In [103]:
import spacy
import re

In [104]:
import string

In [105]:
import pandas as pd

In [106]:
from numpy import dot
from numpy.linalg import norm

In [107]:
from sklearn.metrics import accuracy_score

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [109]:
from gensim.models import KeyedVectors, TranslationMatrix, fasttext
from gensim.test.utils import datapath, temporary_file

In [110]:
#import fasttext

In [111]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
#nlp = spacy.load('en_core_web_lg')
model_en = fasttext.load_facebook_vectors("cc.en.300.bin")
print("en loaded")

In [ ]:
def cosineSimilarity(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [ ]:
#avec = model_en.get_vector('tea')
#bvec = model_en.get_vector('cat')

aavec = model_en['tea']
bbvec = model_en['cat']


cosineSimilarity(aavec,bbvec)

In [ ]:
def getPercentageOfPredicted(a):
    counter = 0
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))

In [ ]:
def getNumberOfOracled(a):
    counter = 0
    
    for x in a:
        if(x[4] != "predicted"):
            counter = counter + 1
            
    return counter

In [ ]:
def getNumberOfPredicted(a):
    counter = 0
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter

In [ ]:
def getPercentageCorrectPredictions(a):
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    counter = counter + len(unclassifiedSamples4GridSearch)
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))

def getLocalAutomationRate(a):
    counter = 0
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(a))

def getLocalAccuracy(a):
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    
            
    return counter / (len(classifiedSamples4GridSearch))

# If beta = 1, then accuracy and automationRate have same importance
# If beta < 1, then accuracy is more important than automation rate
# If beta > 1, then automation rate is more important than accuracy
def getHarmonicScore (accuracy, automationRate, beta):
    
    if(accuracy == 0 and automationRate == 0):
        return 0
    
    score = (1 + beta ** 2) * (accuracy * automationRate) / ((accuracy * (beta ** 2)) + automationRate)
    #s1 = (1 + beta**2)
    #s2 = (accuracy * automationRate)
    #s3 = (accuracy*(beta**2)) + automationRate
    #print(s1)
    #print(s2)
    #print(s3)
    
    return score

In [ ]:
'''
 STRUCTURE OF allSamplesWithLabelThird: ticketText, ticketLabel, TF-IDF, wordVector

 STRUCTURE OF classifiedSamplesInit: ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
'''

In [ ]:
def computeCosineSimilarity(sentence1, sentence2):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    similarity = doc1.similarity(doc2)
    #print(similarity)
    return similarity

In [ ]:
def preprocessing(sentence):
    
    doc = nlp(sentence)
    inputSent = ""

    #removing stop words
    for token in doc:
        
        if(token.is_stop == True):
            continue
        else:
            inputSent = inputSent + " " + token.lemma_
        
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    
    #lower case of the sentence
    inputSent = inputSent.lower()
    #remove numbers
    inputSent = re.sub('\d+', '', inputSent)
    #remover punctuation
    inputSent = inputSent.translate(str.maketrans('', '', string.punctuation))
    #remove multiple whitespaces
    inputSent = ' '.join(inputSent.split())
    
    
    
    #print(inputSent)
    return str(inputSent)

In [ ]:
computeCosineSimilarity("The cat chases the dog", "The dog chases the cat")

In [ ]:
df = pd.read_csv('IBM_shuffled.csv', delimiter=",")

In [64]:
uniqueLabels = df.Label.unique()

foundLabels = []
classifiedSamplesInit = []

tfIdfList = []
allSamplesInit = []
allSamplesWithLabelInit = []

allSamplesMed = []
allSamplesWithLabelMed = []

allSamplesThird = []
allSamplesWithLabelThird = []

allSamplesOracled = []


In [65]:
#Save information from df in variables. Hereby, we compute the tf-idf values and eliminate stop words, numbers and more.

for ind in df.index:
    preprocessedSentence = preprocessing(df['Text'][ind])
    allSamplesWithLabelInit.append([preprocessedSentence, df['Label'][ind]])
    allSamplesInit.append(preprocessedSentence)
    
    

#Get TF-IDF Values
analyze = vectorizer.build_analyzer()
X = vectorizer.fit_transform(allSamplesInit)
(vectorizer.get_feature_names())
tfIdfList = X.toarray()

In [66]:
def getTfIdfSentenceEmbedding(sentence, sentenceCounter):
    
    doc = nlp(str(sentence))
    embedding = 0
    tfIdfForSentence = dict(zip(vectorizer.get_feature_names(), X.toarray()[sentenceCounter]))

    #print(tfIdfForSentence)

    #removing stop words
    counter = 0
    for token in doc:
        
        #print(token.text)
        
        tfIdfValue = 0
        
        try:
            tfIdfValue = tfIdfForSentence[token.text]
            counter = counter + 1
        except:
            pass
            #print("Key not found: ", token.text)
        
        embedding = embedding + (tfIdfValue * nlp(token.text).vector)
    
    embedding = embedding/counter
        
    return embedding
    
    

In [67]:
def getFasttextSentenceEmbedding(sentence, sentenceCounter, useTfIdf):
    doc = nlp(str(sentence))
    embedding = 0
    tfIdfForSentence = dict(zip(vectorizer.get_feature_names(), X.toarray()[sentenceCounter]))

    #print(tfIdfForSentence)

    #removing stop words
    counter = 0
    for token in doc:
        
        #print(token.text)
        
        tfIdfValue = 0
        
        if(useTfIdf):
        
            try:
                tfIdfValue = tfIdfForSentence[token.text]
                counter = counter + 1
                
            except:
                pass
                #print("Key not found: ", token.text)
        else:
            tfIdfValue = 1
            counter = counter + 1
        
        
        embedding = embedding + (tfIdfValue * model_en[token.text])
    if(counter == 0):
        counter = 1
    embedding = embedding / counter
        
    return embedding
    
    

In [68]:
embedding = getTfIdfSentenceEmbedding(allSamplesInit[1], 1)

cosineSimilarity(nlp(allSamplesInit[1]).vector, embedding)

0.9560041

In [69]:
#Set up a list for further work with tuples of the format (text, label, tf-idf-values)
counter = 0
for sample in allSamplesWithLabelInit:
    text = sample[0]
    lable = sample[1]
    tfIdfVal = tfIdfList[counter]
    #embedding = nlp(text).vector 
    #embedding = getTfIdfSentenceEmbedding(text, counter)
    embedding = getFasttextSentenceEmbedding(text, counter, True)
    allSamplesWithLabelMed.append([text, lable, tfIdfVal, embedding])
    
    counter = counter + 1 
    
#The oracle assigns the label for one sample per category    

for x in allSamplesWithLabelMed:
    
    wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[3]]

    if(not(x[1] in foundLabels)):
        foundLabels.append(x[1])
        classifiedSamplesInit.append(wholeEntry)
    else: 
        allSamplesWithLabelThird.append(x)
        
    allSamplesOracled.append(wholeEntry)


sentenceDict = {}
similarityDict = {}
initDict = {}

for label in uniqueLabels:
    sentenceDict.update({label: ""})
    
for label in uniqueLabels:
    similarityDict.update({label: 0})
    initDict.update({label: 0})
    
#print(similarityDict, '\n \n')

In [70]:
len(allSamplesOracled)

400

In [71]:
def initialSampleClassification(minVal):
    
    for x in allSamplesWithLabelMed:
        
        label = x[1]
        numSamples = initDict[label]
        
        if(numSamples < minVal):
            foundLabels.append(x[1])
            wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[3]]
            classifiedSamplesInit.append(wholeEntry)
            
            numSamples = numSamples +1
            
            initDict.update({label: numSamples})
            
        else: 
            allSamplesWithLabelThird.append(x)
    

In [72]:
#initialSampleClassification(4)

In [73]:
#len(allSamplesWithLabelThird)

In [74]:
#len(classifiedSamplesInit)

In [75]:
def addOracledSamples(listOfSamples, samplesPerCategory):
    
    startDict = {}
    
    for label in uniqueLabels:
        startDict.update({label: 0})
        
    classifiedSamples1 = []
        
    limit = samplesPerCategory * len(uniqueLabels)
    
    unclassifiedList = listOfSamples[:]
    
    unclassifiedListFinal = []
    
    #print(limit, len(unclassifiedList))
    #print(startDict)
    
    for x in unclassifiedList:
        #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, embedding
        wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[5]]
        
        
        amountOfSamplesPerLabel = startDict.get(x[1])
        
        if(amountOfSamplesPerLabel < samplesPerCategory):
            #print(x[1], amountOfSamplesPerLabel)
            classifiedSamples1.append(wholeEntry)
            amountOfSamplesPerLabel = amountOfSamplesPerLabel + 1
            startDict.update({x[1]: amountOfSamplesPerLabel})
            
            
            unclassifiedList.remove(x)
            
            
        if(limit == len(classifiedSamples1)):
            break
            
    for x in unclassifiedList:
        text = x[0]
        lable = x[1]
        tfIdfVal = x[2]
        
        unclassifiedListFinal.append([text, lable, tfIdfVal, x[5]])
    
            
            
    print(len(classifiedSamples1), len(unclassifiedListFinal))
    
    #helper = [i[1] for i in classifiedSamples1]
    
    #print(helper)
    
    #print(startDict) 
    
    return classifiedSamples1, unclassifiedListFinal
            
    

In [89]:
def resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2):
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    consideredSamples = allSamplesOracled[startIndex1:endIndex1] + allSamplesOracled[startIndex2:endIndex2]
    #print(consideredSamples[1])
    
    print(len(consideredSamples))
    
    classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(consideredSamples, 2)

In [90]:
classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)

4 196


In [91]:
resetClassifiedSamples(300, 400, 100, 200)

200
8 192


In [92]:
len(classifiedSamples4GridSearch)

8

In [93]:
allUnclassifiedSamples = allSamplesWithLabelThird[:]
allClassifiedSamples = classifiedSamplesInit[:]

In [94]:
#Hereby, if we have 4 categories, we save the biggest similarity per each category with the new sample 
#If the similarites are more or less the same (DEFINE Threshold), then 
def findAndClassifyEqualSimilaritySamples(similarityHP, gapHP):
    
    maxSimilarityPerCategories = []
    
    for label in uniqueLabels:
        maxSimilarityPerCategories.append((label, 0))
    #print(maxSimilarityPerCategories)
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    
    for unclassifiedSample in unclassifiedSamples4GridSearch:
    
        mostSimilarSample = 0
        maxCosSim = 0
        maxCosSimLabel = ""
        
        for label in uniqueLabels:
            similarityDict.update({label: 0})
    
        for classifiedSample in classifiedSamples4GridSearch:
            #cosSim = cosineSimilarity(unclassifiedSample[2], classifiedSample[2])
            #print(len(unclassifiedSample), len(classifiedSample))
            cosSim = cosineSimilarity((unclassifiedSample[3]), (classifiedSample[5]))
            classifiedLabel = classifiedSample[1]
            
            if(maxCosSim < cosSim):
                maxCosSim = cosSim
                maxCosSimLabel = classifiedLabel
            
            if(cosSim > similarityDict[classifiedLabel]):
                similarityDict [classifiedLabel] = cosSim
                sentenceDict [classifiedLabel] = unclassifiedSample[0]
        
        gapPassed = False
        
        for i in similarityDict:
            if(i != maxCosSimLabel):
                gap = maxCosSim - similarityDict[i]
                if(gap < gapHP):
                    gapPassed = True
                    break
        
        if(maxCosSim > similarityHP and gapPassed):
            # ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted
            entry = [unclassifiedSample[0], unclassifiedSample[1], unclassifiedSample[2], unclassifiedSample[1], "oracle", unclassifiedSample[3]]
            #print("entry ", entry)
            unclassifiedSamples4GridSearch.remove(unclassifiedSample)
            classifiedSamples4GridSearch.append(entry)
            
        elif(maxCosSim > similarityHP):
            #STRUCTURE OF classifiedSamplesInit: 
            #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
            entry = [unclassifiedSample[0], maxCosSimLabel, unclassifiedSample[2], unclassifiedSample[1], "predicted", unclassifiedSample[3]]
            #print("entry ", entry)
            unclassifiedSamples4GridSearch.remove(unclassifiedSample)
            classifiedSamples4GridSearch.append(entry)
            
        
            
 
        #print(similarityDict, unclassifiedSample[1])
        #print()
    #print("FINISHED ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
    #print()
    #print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
    #print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
    #print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
    #print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))
    return len(unclassifiedSamples4GridSearch)

In [95]:
def gridSearchAnd3FoldValidation (minSimilarity, maxSimilarity, similarityStep, minGap, maxGap, gapStep, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    similarity = minSimilarity
    
    while(similarity <= maxSimilarity):
        similarityGrid.append(similarity)
        similarity = similarity + similarityStep
        
    g = minGap

    while(g <= maxGap):
        gapGrid.append(g)
        g = g + gapStep
        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
        
    for sim in similarityGrid:
    
        
        for gap in gapGrid:
            
            print("NOW NEW COMBINATION: ", sim, gap)
            
            numberOfPredicted = -1
            numberOfOracled = -1

            localAccuracy = -1
            globalAccuracy = -1
            localAutomationRate = -1
            globalAutomationRate = -1

            localHarmonicScore = -1
            globalHarmonicScore = -1

            unlabelledSamples = 1000
            
            currentSim = sim
            currentGap = gap
            
            simCounter = 0
            gapCounter = 0
            counter = 0
            
            resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            
            while (unlabelledSamples != 0 and counter < 35):
                
                
                #print("sim, gap ", currentSim, currentGap)
                unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)      
            
                numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
                numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
                
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                    currentSim = currentSim - 0.01
                    simCounter = simCounter + 1
                    
                '''
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                    #currentGap = currentGap - 0.005
                    gapCounter = gapCounter + 1
                '''
                
                numberOfOracled = numberOfNewOracled
                numberOfPredicted = numberOfNewPredicted

                if(unlabelledSamples == 0):
                    counter = counter + 10000
                    
                print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
                print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
                print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
                print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))
                
                #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
                counter = counter + 1 
            
            #harmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
            localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
            localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            globalAutomationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            globalAccuracy  = getPercentageCorrectPredictions(classifiedSamples4GridSearch)
            globalHarmonicScore = getHarmonicScore(globalAccuracy, globalAutomationRate, 0.5)
            
            
            if(globalHarmonicScore > maxHarmonicScore):
                maxHarmonicScore = localHarmonicScore
                bestAccuracy = localAccuracy
                bestAutomationRate = localAutomationRate
                bestGap = gap
                bestSim = sim
            
                
            #print("")
            #print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            
            
            
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            #print("")
            
            print(globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap)
            
            tempResults = [globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap]
            allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestGap, bestSim)
    return allResults, dfColumnName
            

In [96]:
def validation (similarityGapGrid, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch

        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
    
    
    
        
    for simGap in similarityGapGrid:
        
        numberOfPredicted = -1
        numberOfOracled = -1
        accuracy = -1
        automationRate = -1
        unlabelledSamples = 1000

        currentSim = simGap[0]
        currentGap = simGap[1]


        simCounter = 0
        gapCounter = 0
        counter = 0
        
        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)

        while (unlabelledSamples != 0 and counter < 100):

            #print("during process ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

            unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)


            numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
            numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
            automationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            accuracy = getPercentageCorrectPredictions(classifiedSamples4GridSearch)

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                currentSim = currentSim - 0.01
                simCounter = simCounter + 1
                #currentGap = currentGap - 1

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                currentGap = currentGap - 0.005
                gapCounter = gapCounter + 1

            numberOfOracled = numberOfNewOracled
            numberOfPredicted = numberOfNewPredicted

            #print("unlab samples ", unlabelledSamples)

            if(unlabelledSamples == 0):
                counter = counter + 10000

            #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
            counter = counter + 1 

        harmonicScore = getHarmonicScore(accuracy, automationRate, 0.5)

        localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
        localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
        localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)

        if(harmonicScore > maxHarmonicScore):
            maxHarmonicScore = harmonicScore
            bestAccuracy = accuracy
            bestAutomationRate = automationRate
            bestSim = simGap[0]
            bestGap = simGap[1]


        #print("")
        #print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
        #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
        #print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
        #print("")

        print(accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1])

        tempResults = [accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1]]
        allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestSim, bestGap)
    return allResults, dfColumnName
            

In [97]:
startIndex1, endIndex1, startIndex2, endIndex2 = 0,400,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = gridSearchAnd3FoldValidation(minSimilarity = 0.90, maxSimilarity = 0.96, similarityStep = 0.05 , minGap = 0.03, maxGap = 0.031, gapStep = 0.01, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

400
8 392
Gap Grid:  [0.03]
Similarity Grid:  [0.9, 0.9500000000000001]


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
NOW NEW COMBINATION:  0.9 0.03
400
8 392
Automation Rate  0.02
Accuracy  0.9875
Number of Predicted  8
Number of Oracled  8
Automation Rate  0.035
Accuracy  0.9825
Number of Predicted  14
Number of Oracled  8
Automation Rate  0.05
Accuracy  0.9775
Number of Predicted  20
Number of Oracled  8
Automation Rate  0.05
Accuracy  0.9775
Number of Predicted  20
Number of Oracled  8
Automation Rate  0.0625
Accuracy  0.9725
Number of Predicted  25
Number of Oracled  9
Automation Rate  0.065
Accuracy  0.9725
Number of Predicted  26
Number of Oracled  9
Automation Rate  0.065
Accuracy  0.9725
Number of Predicted  26
Number of Oracled  9
Automation Rate  0.09
Accuracy  0.9625
Number of Predicted  36
Number of Oracled  9
Automation Rate  0.1375
Accuracy  0.9325
Number of Predicted  55
Nu

In [98]:
startIndex1, endIndex1, startIndex2, endIndex2 = 0,200,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,300,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults2, tfIdfColumns2 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,200,300
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults3, tfIdfColumns3 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)


#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

200
8 192
Gap Grid:  [0.01, 0.015, 0.02, 0.025, 0.030000000000000002, 0.035, 0.04, 0.045, 0.049999999999999996]
Similarity Grid:  [0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35, 0.39999999999999997, 0.44999999999999996, 0.49999999999999994, 0.5499999999999999, 0.6, 0.65, 0.7000000000000001, 0.7500000000000001, 0.8000000000000002, 0.8500000000000002, 0.9000000000000002]


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
NOW NEW COMBINATION:  0.1 0.01
200
8 192
Automation Rate  0.415
Accuracy  0.79
Number of Predicted  83
Number of Oracled  21
Automation Rate  0.61
Accuracy  0.685
Number of Predicted  122
Number of Oracled  30
Automation Rate  0.69
Accuracy  0.65
Number of Predicted  138
Number of Oracled  38
Automation Rate  0.745
Accuracy  0.615
Number of Predicted  149
Number of Oracled  39
Automation Rate  0.775
Accuracy  0.595
Number of Predicted  155
Number of Oracled  39
Automation Rate  

Automation Rate  0.56
Accuracy  0.73
Number of Predicted  112
Number of Oracled  40
Automation Rate  0.64
Accuracy  0.705
Number of Predicted  128
Number of Oracled  48
Automation Rate  0.69
Accuracy  0.67
Number of Predicted  138
Number of Oracled  50
Automation Rate  0.72
Accuracy  0.655
Number of Predicted  144
Number of Oracled  50
Automation Rate  0.73
Accuracy  0.645
Number of Predicted  146
Number of Oracled  51
Automation Rate  0.735
Accuracy  0.64
Number of Predicted  147
Number of Oracled  52
Automation Rate  0.74
Accuracy  0.64
Number of Predicted  148
Number of Oracled  52
0.64 0.74 0.6577777777777779 0.64 0.74 0.6577777777777779 52 148 0.15000000000000002 0.015
NOW NEW COMBINATION:  0.15000000000000002 0.02
200
8 192
Automation Rate  0.345
Accuracy  0.86
Number of Predicted  69
Number of Oracled  35
Automation Rate  0.52
Accuracy  0.79
Number of Predicted  104
Number of Oracled  48
Automation Rate  0.59
Accuracy  0.78
Number of Predicted  118
Number of Oracled  58
Automati

Automation Rate  0.52
Accuracy  0.79
Number of Predicted  104
Number of Oracled  48
Automation Rate  0.59
Accuracy  0.78
Number of Predicted  118
Number of Oracled  58
Automation Rate  0.635
Accuracy  0.755
Number of Predicted  127
Number of Oracled  61
Automation Rate  0.66
Accuracy  0.745
Number of Predicted  132
Number of Oracled  62
Automation Rate  0.67
Accuracy  0.735
Number of Predicted  134
Number of Oracled  63
Automation Rate  0.675
Accuracy  0.735
Number of Predicted  135
Number of Oracled  64
Automation Rate  0.68
Accuracy  0.735
Number of Predicted  136
Number of Oracled  64
0.735 0.68 0.7232995658465992 0.735 0.68 0.7232995658465992 64 136 0.2 0.02
NOW NEW COMBINATION:  0.2 0.025
200
8 192
Automation Rate  0.32
Accuracy  0.875
Number of Predicted  64
Number of Oracled  40
Automation Rate  0.49
Accuracy  0.805
Number of Predicted  98
Number of Oracled  54
Automation Rate  0.56
Accuracy  0.795
Number of Predicted  112
Number of Oracled  64
Automation Rate  0.6
Accuracy  0.7

Automation Rate  0.45
Accuracy  0.82
Number of Predicted  90
Number of Oracled  62
Automation Rate  0.515
Accuracy  0.81
Number of Predicted  103
Number of Oracled  73
Automation Rate  0.55
Accuracy  0.795
Number of Predicted  110
Number of Oracled  78
Automation Rate  0.565
Accuracy  0.79
Number of Predicted  113
Number of Oracled  81
Automation Rate  0.565
Accuracy  0.79
Number of Predicted  113
Number of Oracled  84
Automation Rate  0.57
Accuracy  0.79
Number of Predicted  114
Number of Oracled  85
Automation Rate  0.57
Accuracy  0.79
Number of Predicted  114
Number of Oracled  86
0.79 0.57 0.7333876221498372 0.79 0.57 0.7333876221498372 86 114 0.25 0.030000000000000002
NOW NEW COMBINATION:  0.25 0.035
200
8 192
Automation Rate  0.24
Accuracy  0.895
Number of Predicted  48
Number of Oracled  56
Automation Rate  0.39
Accuracy  0.83
Number of Predicted  78
Number of Oracled  74
Automation Rate  0.45
Accuracy  0.82
Number of Predicted  90
Number of Oracled  86
Automation Rate  0.485
Ac

Automation Rate  0.33
Accuracy  0.85
Number of Predicted  66
Number of Oracled  86
Automation Rate  0.385
Accuracy  0.835
Number of Predicted  77
Number of Oracled  99
Automation Rate  0.42
Accuracy  0.82
Number of Predicted  84
Number of Oracled  104
Automation Rate  0.435
Accuracy  0.815
Number of Predicted  87
Number of Oracled  107
Automation Rate  0.435
Accuracy  0.815
Number of Predicted  87
Number of Oracled  110
Automation Rate  0.44
Accuracy  0.815
Number of Predicted  88
Number of Oracled  111
Automation Rate  0.44
Accuracy  0.815
Number of Predicted  88
Number of Oracled  112
0.815 0.44 0.6963106796116504 0.815 0.44 0.6963106796116504 112 88 0.3 0.04
NOW NEW COMBINATION:  0.3 0.045
200
8 192
Automation Rate  0.18
Accuracy  0.915
Number of Predicted  36
Number of Oracled  68
Automation Rate  0.28
Accuracy  0.865
Number of Predicted  56
Number of Oracled  96
Automation Rate  0.335
Accuracy  0.85
Number of Predicted  67
Number of Oracled  109
Automation Rate  0.365
Accuracy  0.

...

Automation Rate  0.31
Accuracy  0.805
Number of Predicted  62
Number of Oracled  35
Automation Rate  0.315
Accuracy  0.8
Number of Predicted  63
Number of Oracled  36
Automation Rate  0.315
Accuracy  0.8
Number of Predicted  63
Number of Oracled  36
Automation Rate  0.34
Accuracy  0.785
Number of Predicted  68
Number of Oracled  44
Automation Rate  0.365
Accuracy  0.77
Number of Predicted  73
Number of Oracled  45
Automation Rate  0.365
Accuracy  0.77
Number of Predicted  73
Number of Oracled  45
Automation Rate  0.37
Accuracy  0.765
Number of Predicted  74
Number of Oracled  50
Automation Rate  0.405
Accuracy  0.74
Number of Predicted  81
Number of Oracled  52
Automation Rate  0.415
Accuracy  0.73
Number of Predicted  83
Number of Oracled  53
Automation Rate  0.415
Accuracy  0.73
Number of Predicted  83
Number of Oracled  53
Automation Rate  0.425
Accuracy  0.72
Number of Predicted  85
Number of Oracled  58
Automation Rate  0.425
Accuracy  0.72
Number of Predicted  85
Number of Oracle

Automation Rate  0.02
Accuracy  0.99
Number of Predicted  4
Number of Oracled  8
Automation Rate  0.025
Accuracy  0.985
Number of Predicted  5
Number of Oracled  8
Automation Rate  0.025
Accuracy  0.985
Number of Predicted  5
Number of Oracled  8
Automation Rate  0.045
Accuracy  0.97
Number of Predicted  9
Number of Oracled  8
Automation Rate  0.095
Accuracy  0.92
Number of Predicted  19
Number of Oracled  8
Automation Rate  0.14
Accuracy  0.875
Number of Predicted  28
Number of Oracled  8
Automation Rate  0.14
Accuracy  0.875
Number of Predicted  28
Number of Oracled  8
Automation Rate  0.175
Accuracy  0.86
Number of Predicted  35
Number of Oracled  8
Automation Rate  0.18
Accuracy  0.86
Number of Predicted  36
Number of Oracled  8
Automation Rate  0.185
Accuracy  0.86
Number of Predicted  37
Number of Oracled  8
Automation Rate  0.185
Accuracy  0.86
Number of Predicted  37
Number of Oracled  8
Automation Rate  0.225
Accuracy  0.835
Number of Predicted  45
Number of Oracled  8
Automat

Automation Rate  0.45
Accuracy  0.7
Number of Predicted  90
Number of Oracled  22
Automation Rate  0.48
Accuracy  0.675
Number of Predicted  96
Number of Oracled  22
Automation Rate  0.48
Accuracy  0.675
Number of Predicted  96
Number of Oracled  22
Automation Rate  0.5
Accuracy  0.66
Number of Predicted  100
Number of Oracled  24
Automation Rate  0.54
Accuracy  0.635
Number of Predicted  108
Number of Oracled  25
Automation Rate  0.555
Accuracy  0.62
Number of Predicted  111
Number of Oracled  25
Automation Rate  0.555
Accuracy  0.62
Number of Predicted  111
Number of Oracled  25
Automation Rate  0.57
Accuracy  0.61
Number of Predicted  114
Number of Oracled  29
0.61 0.57 0.6015570934256055 0.45454545454545453 0.7972027972027972 0.4972954109230501 29 114 0.9000000000000002 0.02
NOW NEW COMBINATION:  0.9000000000000002 0.025
200
8 192
Automation Rate  0.01
Accuracy  1.0
Number of Predicted  2
Number of Oracled  8
Automation Rate  0.01
Accuracy  1.0
Number of Predicted  2
Number of Orac

Automation Rate  0.23
Accuracy  0.865
Number of Predicted  46
Number of Oracled  21
Automation Rate  0.25
Accuracy  0.855
Number of Predicted  50
Number of Oracled  22
Automation Rate  0.25
Accuracy  0.855
Number of Predicted  50
Number of Oracled  22
Automation Rate  0.27
Accuracy  0.84
Number of Predicted  54
Number of Oracled  28
Automation Rate  0.285
Accuracy  0.825
Number of Predicted  57
Number of Oracled  30
Automation Rate  0.295
Accuracy  0.825
Number of Predicted  59
Number of Oracled  30
Automation Rate  0.3
Accuracy  0.82
Number of Predicted  60
Number of Oracled  30
Automation Rate  0.3
Accuracy  0.82
Number of Predicted  60
Number of Oracled  30
Automation Rate  0.325
Accuracy  0.8
Number of Predicted  65
Number of Oracled  32
Automation Rate  0.33
Accuracy  0.795
Number of Predicted  66
Number of Oracled  33
Automation Rate  0.33
Accuracy  0.795
Number of Predicted  66
Number of Oracled  33
Automation Rate  0.36
Accuracy  0.775
Number of Predicted  72
Number of Oracled 

Automation Rate  0.04
Accuracy  0.99
Number of Predicted  8
Number of Oracled  9
Automation Rate  0.07
Accuracy  0.965
Number of Predicted  14
Number of Oracled  13
Automation Rate  0.105
Accuracy  0.955
Number of Predicted  21
Number of Oracled  15
Automation Rate  0.105
Accuracy  0.955
Number of Predicted  21
Number of Oracled  15
Automation Rate  0.13
Accuracy  0.95
Number of Predicted  26
Number of Oracled  17
Automation Rate  0.135
Accuracy  0.95
Number of Predicted  27
Number of Oracled  17
Automation Rate  0.14
Accuracy  0.95
Number of Predicted  28
Number of Oracled  17
Automation Rate  0.14
Accuracy  0.95
Number of Predicted  28
Number of Oracled  17
Automation Rate  0.16
Accuracy  0.95
Number of Predicted  32
Number of Oracled  21
Automation Rate  0.165
Accuracy  0.95
Number of Predicted  33
Number of Oracled  25
Automation Rate  0.165
Accuracy  0.95
Number of Predicted  33
Number of Oracled  25
Automation Rate  0.18
Accuracy  0.945
Number of Predicted  36
Number of Oracled  

In [99]:
getLocalAccuracy(classifiedSamples4GridSearch)
len(classifiedSamples4GridSearch)

143

In [100]:
from pandas import DataFrame

In [101]:
df1 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)
df2 = DataFrame (tfIdfResults2, columns= tfIdfColumns2)
df3 = DataFrame (tfIdfResults3, columns= tfIdfColumns3)
df1

,Accuracy,Automation Rate,Harmonic Score,Local Accuracy,Local Automation Rate,Local Harmonic Score,Number Of Oracled,Number Of Predicted,Minimal Similarity,Minimal Gap
0,0.575,0.795,0.608688,0.575000,0.795000,0.608688,41,159,0.1,0.010
1,0.640,0.740,0.657778,0.640000,0.740000,0.657778,52,148,0.1,0.015
2,0.735,0.680,0.723300,0.735000,0.680000,0.723300,64,136,0.1,0.020
3,0.765,0.630,0.733562,0.765000,0.630000,0.733562,74,126,0.1,0.025
4,0.790,0.570,0.733388,0.790000,0.570000,0.733388,86,114,0.1,0.030
...,...,...,...,...,...,...,...,...,...,...
148,0.800,0.515,0.720280,0.724138,0.710345,0.721337,42,103,0.9,0.030
149,0.815,0.490,0.719550,0.744828,0.675862,0.729931,47,98,0.9,0.035
150,0.860,0.425,0.713867,0.806897,0.586207,0.750396,60,85,0.9,0.040
151,0.865,0.405,0.704879,0.813793,0.558621,0.745670,64,81,0.9,0.045


In [102]:
with pd.ExcelWriter("fastText_2PerCat-WITH_tf_idf-CFV.xlsx") as writer:
    df1.to_excel(writer, sheet_name='1', index=False)
    df2.to_excel(writer, sheet_name='2', index=False)
    df3.to_excel(writer, sheet_name='3', index=False)

In [117]:
startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0
#to be changed
simGapGrid1 = [[0.85, 0.015], [0.85, 0.01]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults11, tfIdfColumns11 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,0,0
#to be changed
simGapGrid2 = [[0.9, 0.03], [0.75, 0.03]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults22, tfIdfColumns22 = validation(simGapGrid2, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,200,0,0
#to be changed
simGapGrid3 = [[0.85, 0.015]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults33, tfIdfColumns33 = validation(simGapGrid3, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96
100
4 96
0.66 0.58 0.6422818791946309 0.66 0.58 0.6422818791946309 42 58 0.85 0.015
100
4 96
100
4 96
0.46 0.75 0.49855491329479773 0.46 0.75 0.49855491329479773 25 75 0.85 0.01
Best:  0.6422818791946309 0.66 0.58 0.85 0.015
100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96
100
4 96
0.82 0.39 0.6718487394957983 0.82 0.39 0.6718487394957983 61 39 0.9 0.03
100
4 96
100
4 96
0.84 0.29 0.609 0.84 0.29 0.609 71 29 0.75 0.03
Best:  0.6718487394957983 0.82 0.39 0.9 0.03
100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap


In [118]:
df11 = DataFrame (tfIdfResults11, columns= tfIdfColumns11)
df22 = DataFrame (tfIdfResults22, columns= tfIdfColumns22)
df33 = DataFrame (tfIdfResults33, columns= tfIdfColumns33)

In [119]:
with pd.ExcelWriter("fastText_WITHOUT_tf_idf-Validation.xlsx") as writer:
    df11.to_excel(writer, sheet_name='1', index=False)
    df22.to_excel(writer, sheet_name='2', index=False)
    df33.to_excel(writer, sheet_name='3', index=False)

In [123]:

startIndex1, endIndex1, startIndex2, endIndex2 = 300,400,0,0
#to be changed
simGapGrid1 = [[0.85, 0.015], [0.9, 0.03]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults111, tfIdfColumns111 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)


import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96
100
4 96
0.29 0.95 0.3367970660146699 0.29 0.95 0.3367970660146699 5 95 0.85 0.015
100
4 96
100
4 96
0.68 0.51 0.6375000000000001 0.68 0.51 0.6375000000000001 49 51 0.9 0.03
Best:  0.6375000000000001 0.68 0.51 0.9 0.03


In [124]:
df111 = DataFrame (tfIdfResults111, columns= tfIdfColumns111)

In [125]:
with pd.ExcelWriter("fastText_WITHOUT_tf_idf-Final.xlsx") as writer:
    df111.to_excel(writer, sheet_name='1', index=False)
    

In [146]:
#findAndClassifyEqualSimilaritySamples()

In [170]:
resetClassifiedSamples(0,200,0,0)

200
4 196


In [176]:
similarityHP = 0.85
gapHP = 0.03
#resetClassifiedSamples(0,200,0,0)
findAndClassifyEqualSimilaritySamples(similarityHP, gapHP)

164

In [177]:
getPercentageOfPredicted(allClassifiedSamples)

0.0

In [178]:
getPercentageCorrectPredictions(allClassifiedSamples)

0.84

In [179]:
getNumberOfPredicted(allClassifiedSamples)

0

In [180]:
getNumberOfOracled(allClassifiedSamples)

4